In [1]:
import mido
import time
import numpy as np

def normalize(P):
    K = P.shape[0]
    norm = np.sum(P, axis=0, keepdims=False)
    return P / norm

# input the file name
midi_string = 'sample1'
midi_file = mido.MidiFile('%s.mid' % midi_string)

# play audio
play_audio = False

current_tempo = 0
time_signature_numerator = 0
time_signature_denominator = 0
time_signature_quarter = 0
ticks_per_beat = midi_file.ticks_per_beat
print('ticks per beat: %d' % ticks_per_beat)

# bar_duration: reset every bar (unit: ticks)
# total_duration: current cummulative duration (unit: seconds)
bar_duration = 0
total_duration = 0

# notes_vec: store the bar duration of the 128 notes in each 16 channels
# notes_vec_on: 'True' is note on, 'False' is note off (or velocity = 0)
notes_vec = np.zeros([16, 128], dtype=np.float16)
notes_vec_on = np.zeros([16, 128], dtype=np.bool)

# use 'print(mido.get_output_names())' to get your port's name and type it in the 'mido.open_output' bracket
port = None
port = mido.open_output('Microsoft GS Wavetable Synth 0')

# turn numeric notes into english notes, octave info not preserved
def note_int2char(note):
    return {
        '0': 'C',
        '1': 'Db',
        '2': 'D',
        '3': 'Eb',
        '4': 'E',
        '5': 'F',
        '6': 'F#',
        '7': 'G',
        '8': 'Ab',
        '9': 'A',
        '10': 'Bb',
        '11': 'B'
    }.get(str(note % 12), 'undef')

# define the chords and its relative position
chords = np.array(
    [
        # Root note only
        [4, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, 4, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, 4, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, 4, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, 4, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, 4, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, 4, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, 4, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, 4, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, 4, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 4, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 4],
        # Power chords
        [4, -1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1],
        [-1, 4, -1, -1, -1, -1, -1, -1, 2, -1, -1, -1],
        [-1, -1, 4, -1, -1, -1, -1, -1, -1, 2, -1, -1],
        [-1, -1, -1, 4, -1, -1, -1, -1, -1, -1, 2, -1],
        [-1, -1, -1, -1, 4, -1, -1, -1, -1, -1, -1, 2],
        [2, -1, -1, -1, -1, 4, -1, -1, -1, -1, -1, -1],
        [-1, 2, -1, -1, -1, -1, 4, -1, -1, -1, -1, -1],
        [-1, -1, 2, -1, -1, -1, -1, 4, -1, -1, -1, -1],
        [-1, -1, -1, 2, -1, -1, -1, -1, 4, -1, -1, -1],
        [-1, -1, -1, -1, 2, -1, -1, -1, -1, 4, -1, -1],
        [-1, -1, -1, -1, -1, 2, -1, -1, -1, -1, 4, -1],
        [-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, 4],
        # Major chords
        [4, -1, -1, -1, 2, -1, -1, 2, -1, -1, -1, -1],
        [-1, 4, -1, -1, -1, 2, -1, -1, 2, -1, -1, -1],
        [-1, -1, 4, -1, -1, -1, 2, -1, -1, 2, -1, -1],
        [-1, -1, -1, 4, -1, -1, -1, 2, -1, -1, 2, -1],
        [-1, -1, -1, -1, 4, -1, -1, -1, 2, -1, -1, 2],
        [2, -1, -1, -1, -1, 4, -1, -1, -1, 2, -1, -1],
        [-1, 2, -1, -1, -1, -1, 4, -1, -1, -1, 2, -1],
        [-1, -1, 2, -1, -1, -1, -1, 4, -1, -1, -1, 2],
        [2, -1, -1, 2, -1, -1, -1, -1, 4, -1, -1, -1],
        [-1, 2, -1, -1, 2, -1, -1, -1, -1, 4, -1, -1],
        [-1, -1, 2, -1, -1, 2, -1, -1, -1, -1, 4, -1],
        [-1, -1, -1, 2, -1, -1, 2, -1, -1, -1, -1, 4],
        # Minor chords
        [4, -1, -1, 2, -1, -1, -1, 2, -1, -1, -1, -1],
        [-1, 4, -1, -1, 2, -1, -1, -1, 2, -1, -1, -1],
        [-1, -1, 4, -1, -1, 2, -1, -1, -1, 2, -1, -1],
        [-1, -1, -1, 4, -1, -1, 2, -1, -1, -1, 2, -1],
        [-1, -1, -1, -1, 4, -1, -1, 2, -1, -1, -1, 2],
        [2, -1, -1, -1, -1, 4, -1, -1, 2, -1, -1, -1],
        [-1, 2, -1, -1, -1, -1, 4, -1, -1, 2, -1, -1],
        [-1, -1, 2, -1, -1, -1, -1, 4, -1, -1, 2, -1],
        [-1, -1, -1, 2, -1, -1, -1, -1, 4, -1, -1, 2],
        [2, -1, -1, -1, 2, -1, -1, -1, -1, 4, -1, -1],
        [-1, 2, -1, -1, -1, 2, -1, -1, -1, -1, 4, -1],
        [-1, -1, 2, -1, -1, -1, 2, -1, -1, -1, -1, 4],
        # Dominant 7th chords
        [3, -1, -1, -1, 1.5, -1, -1, 1.5, -1, -1, 2, -1],
        [-1, 3, -1, -1, -1, 1.5, -1, -1, 1.5, -1, -1, 2],
        [2, -1, 3, -1, -1, -1, 1.5, -1, -1, 1.5, -1, -1],
        [-1, 2, -1, 3, -1, -1, -1, 1.5, -1, -1, 1.5, -1],
        [-1, -1, 2, -1, 3, -1, -1, -1, 1.5, -1, -1, 1.5],
        [1.5, -1, -1, 2, -1, 3, -1, -1, -1, 1.5, -1, -1],
        [-1, 1.5, -1, -1, 2, -1, 3, -1, -1, -1, 1.5, -1],
        [-1, -1, 1.5, -1, -1, 2, -1, 3, -1, -1, -1, 1.5],
        [1.5, -1, -1, 1.5, -1, -1, 2, -1, 3, -1, -1, -1],
        [-1, 1.5, -1, -1, 1.5, -1, -1, 2, -1, 3, -1, -1],
        [-1, -1, 1.5, -1, -1, 1.5, -1, -1, 2, -1, 3, -1],
        [-1, -1, -1, 1.5, -1, -1, 1.5, -1, -1, 2, -1, 3],
        # Major 7th chords
        [3, -1, -1, -1, 1.5, -1, -1, 1.5, -1, -1, -1, 2],
        [2, 3, -1, -1, -1, 1.5, -1, -1, 1.5, -1, -1, -1],
        [-1, 2, 3, -1, -1, -1, 1.5, -1, -1, 1.5, -1, -1],
        [-1, -1, 2, 3, -1, -1, -1, 1.5, -1, -1, 1.5, -1],
        [-1, -1, -1, 2, 3, -1, -1, -1, 1.5, -1, -1, 1.5],
        [1.5, -1, -1, -1, 2, 3, -1, -1, -1, 1.5, -1, -1],
        [-1, 1.5, -1, -1, -1, 2, 3, -1, -1, -1, 1.5, -1],
        [-1, -1, 1.5, -1, -1, -1, 2, 3, -1, -1, -1, 1.5],
        [1.5, -1, -1, 1.5, -1, -1, -1, 2, 3, -1, -1, -1],
        [-1, 1.5, -1, -1, 1.5, -1, -1, -1, 2, 3, -1, -1],
        [-1, -1, 1.5, -1, -1, 1.5, -1, -1, -1, 2, 3, -1],
        [-1, -1, -1, 1.5, -1, -1, 1.5, -1, -1, -1, 2, 3],
        # Minor 7th chords
        [3, -1, -1, 1.5, -1, -1, -1, 1.5, -1, -1, 2, -1],
        [-1, 3, -1, -1, 1.5, -1, -1, -1, 1.5, -1, -1, 2],
        [2, -1, 3, -1, -1, 1.5, -1, -1, -1, 1.5, -1, -1],
        [-1, 2, -1, 3, -1, -1, 1.5, -1, -1, -1, 1.5, -1],
        [-1, -1, 2, -1, 3, -1, -1, 1.5, -1, -1, -1, 1.5],
        [1.5, -1, -1, 2, -1, 3, -1, -1, 1.5, -1, -1, -1],
        [-1, 1.5, -1, -1, 2, -1, 3, -1, -1, 1.5, -1, -1],
        [-1, -1, 1.5, -1, -1, 2, -1, 3, -1, -1, 1.5, -1],
        [-1, -1, -1, 1.5, -1, -1, 2, -1, 3, -1, -1, 1.5],
        [1.5, -1, -1, -1, 1.5, -1, -1, 2, -1, 3, -1, -1],
        [-1, 1.5, -1, -1, -1, 1.5, -1, -1, 2, -1, 3, -1],
        [-1, -1, 1.5, -1, -1, -1, 1.5, -1, -1, 2, -1, 3],
        # Diminished chords
        [4, -1, -1, 2, -1, -1, 2, -1, -1, -1, -1, -1],
        [-1, 4, -1, -1, 2, -1, -1, 2, -1, -1, -1, -1],
        [-1, -1, 4, -1, -1, 2, -1, -1, 2, -1, -1, -1],
        [-1, -1, -1, 4, -1, -1, 2, -1, -1, 2, -1, -1],
        [-1, -1, -1, -1, 4, -1, -1, 2, -1, -1, 2, -1],
        [-1, -1, -1, -1, -1, 4, -1, -1, 2, -1, -1, 2],
        [2, -1, -1, -1, -1, -1, 4, -1, -1, 2, -1, -1],
        [-1, 2, -1, -1, -1, -1, -1, 4, -1, -1, 2, -1],
        [-1, -1, 2, -1, -1, -1, -1, -1, 4, -1, -1, 2],
        [2, -1, -1, 2, -1, -1, -1, -1, -1, 4, -1, -1],
        [-1, 2, -1, -1, 2, -1, -1, -1, -1, -1, 4, -1],
        [-1, -1, 2, -1, -1, 2, -1, -1, -1, -1, -1, 4],
    ], dtype=np.int8)

# name the chord
def name_chord(chord):
    return {
        '0': 'C1',
        '1': 'Db1',
        '2': 'D1',
        '3': 'Eb1',
        '4': 'E1',
        '5': 'F1',
        '6': 'F#1',
        '7': 'G1',
        '8': 'Ab1',
        '9': 'A1',
        '10': 'Bb1',
        '11': 'B1',
        '12': 'C5',
        '13': 'Db5',
        '14': 'D5',
        '15': 'Eb5',
        '16': 'E5',
        '17': 'F5',
        '18': 'F#5',
        '19': 'G5',
        '20': 'Ab5',
        '21': 'A5',
        '22': 'Bb5',
        '23': 'B5',
        '24': 'C',
        '25': 'Db',
        '26': 'D',
        '27': 'Eb',
        '28': 'E',
        '29': 'F',
        '30': 'F#',
        '31': 'G',
        '32': 'Ab',
        '33': 'A',
        '34': 'Bb',
        '35': 'B',
        '36': 'Cm',
        '37': 'Dbm',
        '38': 'Dm',
        '39': 'Ebm',
        '40': 'Em',
        '41': 'Fm',
        '42': 'F#m',
        '43': 'Gm',
        '44': 'Abm',
        '45': 'Am',
        '46': 'Bbm',
        '47': 'Bm',
        '48': 'C7',
        '49': 'Db7',
        '50': 'D7',
        '51': 'Eb7',
        '52': 'E7',
        '53': 'F7',
        '54': 'F#7',
        '55': 'G7',
        '56': 'Ab7',
        '57': 'A7',
        '58': 'Bb7',
        '59': 'B7',
        '60': 'CM7',
        '61': 'DbM7',
        '62': 'DM7',
        '63': 'EbM7',
        '64': 'EM7',
        '65': 'FM7',
        '66': 'F#M7',
        '67': 'GM7',
        '68': 'AbM7',
        '69': 'AM7',
        '70': 'BbM7',
        '71': 'BM7',
        '72': 'Cm7',
        '73': 'Dbm7',
        '74': 'Dm7',
        '75': 'Ebm7',
        '76': 'Em7',
        '77': 'Fm7',
        '78': 'F#m7',
        '79': 'Gm7',
        '80': 'Abm7',
        '81': 'Am7',
        '82': 'Bbm7',
        '83': 'Bm7',
        '84': 'Cdim',
        '85': 'Dbdim',
        '86': 'Ddim',
        '87': 'Ebdim',
        '88': 'Edim',
        '89': 'Fdim',
        '90': 'F#dim',
        '91': 'Gdim',
        '92': 'Abdim',
        '93': 'Adim',
        '94': 'Bbdim',
        '95': 'Bdim',
    }.get(str(chord), 'undef')

# chords to key, '[C, Db, D, ..., B, Cm, Dbm, ..., Bbm, Bm]'
chord_to_key = np.array(
    [
        # Major chords
        [2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0],
        [0, 0, 2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1],
        [0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
        [1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
        [0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1],
        [1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1],
        [0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0],
        # Minor chords
        [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 1, 0],
        [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 1],
        [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2],
        # Dominant 7th chords
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        # Major 7th chords
        [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
        # Minor 7th chords
        [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
        [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1],
        [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
        # Diminished chords
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    ], dtype=np.int8)

# name the key
def name_key(key):
    return {
        '0': 'C Major',
        '1': 'Db(C#) Major',
        '2': 'D Major',
        '3': 'Eb(D#) Major',
        '4': 'E Major',
        '5': 'F Major',
        '6': 'F#(Gb) Major',
        '7': 'G Major',
        '8': 'Ab(G#) Major',
        '9': 'A Major',
        '10': 'Bb(A#) Major',
        '11': 'B Major',
        '12': 'C minor',
        '13': 'Db(C#) minor',
        '14': 'D minor',
        '15': 'Eb(D#) minor',
        '16': 'E minor',
        '17': 'F minor',
        '18': 'F#(Gb) minor',
        '19': 'G minor',
        '20': 'Ab(G#) minor',
        '21': 'A minor',
        '22': 'Bb(A#) minor',
        '23': 'B minor',
    }.get(str(key), 'undef')

# get chords of a key
def key_to_chords(key):
    return np.nonzero(np.transpose(chord_to_key)[np.argmax(key)])

chords_stat = np.zeros(chords.shape[0], dtype=np.uint16)
chords_all = []

try:

    for msg in midi_file:
        
        # delay for playing
        if play_audio:
            time.sleep(msg.time)

        if msg.time != 0:

            total_duration += msg.time
            bar_duration += mido.second2tick(msg.time, ticks_per_beat, current_tempo)

            # indication of 'bar_duration' reaching the duration of a complete bar
            if bar_duration / ticks_per_beat >= time_signature_quarter:

                # break the continue-over-bars note-on notes
                notes_vec[np.where(notes_vec_on)] += total_duration
                
                # reset 'bar_duration'
                bar_duration_prev = bar_duration
                bar_duration = bar_duration - ticks_per_beat * time_signature_quarter * (bar_duration // (ticks_per_beat * time_signature_quarter))
                
                # find out the played notes of the complete bar
                played_notes_idx = np.where(notes_vec != 0)
                if played_notes_idx[0].size > 0:
                    played_notes = np.zeros(12, dtype=np.float16)
                    for played_note_idx in np.nditer(played_notes_idx):
                        played_notes[played_note_idx[1] % 12] += notes_vec[played_note_idx[0]][played_note_idx[1]]
                    played_notes = normalize(played_notes)
                                        
                    # give a better 'print' for played notes (all values are from 'played_notes')
                    played_notes_print = [np.where(played_notes != 0)[0], played_notes[played_notes != 0]]
                    played_notes_printf = ''
                    for played_note_print in np.nditer(played_notes_print):
                        played_notes_printf += '{:<2}'.format(note_int2char(played_note_print[0])) + ': ' + "{0:.2f}".format(played_note_print[1]) + '  '
                    
                    # calculate the chord similarity score
                    chords_score = np.dot(chords, played_notes)
                    chord_predict = np.argmax(chords_score)
                    chords_stat[chord_predict] += 1
                    
                    # print the chord analysis result
                    print('%s\t%s' % (name_chord(chord_predict), played_notes_printf))
                    
                    chords_all.append([total_duration, chord_predict])
                        
                # reset the notes_vec and notes_vec_on for next bar
                notes_vec = np.zeros([16, 128], dtype=np.float16)
                notes_vec_on = np.zeros([16, 128], dtype=np.bool)
                
                # continue the continue-over-bars-note-on notes
                notes_vec[np.where(notes_vec_on)] -= total_duration
                
                # debug message for bar detection
                # print('next bar:\t%.2f to %.2f' % (bar_duration_prev, bar_duration))
                # print(total_duration)

        if msg.is_meta:

            # change time signature should be the first beat
            if msg.type == 'time_signature':
                time_signature_denominator = msg.denominator
                time_signature_numerator = msg.numerator
                time_signature_quarter = time_signature_numerator / (time_signature_denominator / 4)
                bar_duration = 0
                print('time signature:\t%d/%d (quarters: %d)' % (time_signature_numerator, time_signature_denominator, time_signature_quarter))

            # update tempo for 'second2tick'
            if msg.type == 'set_tempo':
                current_tempo = msg.tempo
                print("set tempo:\t%d (bpm: %.2f)" % (msg.tempo, mido.tempo2bpm(msg.tempo)))

        if not msg.is_meta:

            # play the note
            if play_audio:
                port.send(msg)
            
            if msg.type == 'note_on':
                
                # ignore drum channel (midi channel = 10)
                if msg.channel != 9:
            
                    if msg.velocity != 0:
                        if not notes_vec_on[msg.channel][msg.note]:
                            notes_vec[msg.channel][msg.note] -= total_duration
                            notes_vec_on[msg.channel][msg.note] = True
                    else:
                        if notes_vec_on[msg.channel][msg.note]:
                            notes_vec[msg.channel][msg.note] += total_duration
                            notes_vec_on[msg.channel][msg.note] = False

            if msg.type == 'note_off':
                
                # ignore drum channel (midi channel = 10)
                if msg.channel != 9:
                    
                    if notes_vec_on[msg.channel][msg.note]:
                        notes_vec[msg.channel][msg.note] += total_duration
                        notes_vec_on[msg.channel][msg.note] = False

        # you can set auto stop in here (seconds as unit)
        if total_duration == -1:

            if port is not None:
                port.close()
            break

    if port is not None:
        port.close()
    print('End of midi')
    key_stat = np.dot(chords_stat[24:], chord_to_key)
    key_predict = np.argmax(key_stat)
    key_acc = np.sum(chords_stat[np.add(key_to_chords(key_stat), 24)]) / np.sum(chords_stat[24:])
    print('\nKey: %s (accuracy: %f)' % (name_key(key_predict), key_acc))
    print('\nChord statistics')
    for chord_stat_idx, chord_stat in enumerate(chords_stat):
        if chord_stat != 0:
            print('%s\t: %d' % (name_chord(chord_stat_idx), chord_stat))
        
# action after pressing stop
except KeyboardInterrupt:
    
    if port is not None:
        port.close()
    print('Forced stop')

ticks per beat: 480
set tempo:	472440 (bpm: 127.00)
time signature:	4/4 (quarters: 4)
D1	D : 1.00  
D1	D : 1.00  
D1	D : 1.00  
D1	D : 1.00  
D1	D : 1.00  
D5	D : 0.49  A : 0.51  
Dm	D : 0.49  F : 0.25  A : 0.25  
Gm	C : 0.03  G : 0.50  Bb: 0.47  
F	F : 0.49  A : 0.51  
A5	E : 0.26  A : 0.74  
A	Db: 0.25  A : 0.74  Bb: 0.01  
Gm	C : 0.04  G : 0.58  Bb: 0.38  
A	Db: 0.16  D : 0.08  F : 0.25  A : 0.50  
Dm	D : 0.62  F : 0.12  A : 0.25  
Dm	D : 0.75  F : 0.13  A : 0.11  Bb: 0.01  
Gm	C : 0.02  G : 0.59  A : 0.01  Bb: 0.37  
F	F : 0.25  Ab: 0.25  A : 0.51  
A	Db: 0.13  E : 0.13  A : 0.75  
A	Db: 0.12  E : 0.13  G : 0.01  A : 0.74  
G1	E : 0.03  F : 0.03  G : 0.93  
F5	C : 0.07  Db: 0.14  D : 0.14  F : 0.65  
D1	D : 1.00  
D1	D : 1.00  
D1	D : 1.00  
D1	D : 1.00  
D1	Db: 0.02  D : 0.98  
Dm	D : 0.90  F : 0.10  
G5	D : 0.01  G : 0.99  
F	F : 0.61  Ab: 0.02  A : 0.37  
A1	Ab: 0.03  A : 0.97  
A1	A : 1.00  
Gm	F : 0.01  G : 0.67  A : 0.03  Bb: 0.29  
Dm	C : 0.13  Db: 0.18  D : 0.23  F : 0.28  

In [33]:
import itertools

def is_note(msg):
    return msg.type == 'note_on' or msg.type == 'note_off'

def is_note_on(msg):
    return msg.type == 'note_on' and msg.velocity != 0

def is_drum(msg):
    return msg.channel == 9

def get_module_tuples(prev_key, new_key):
    module_tuples = np.zeros((128, 2), dtype=np.int8)

    for i in range(128):
        module_tuples[i, 0] = i
    
    tonic_note = prev_key % 12
    prev_key_type = prev_key // 12
    new_key_type = new_key // 12
    
    if prev_key_type == 0:
        # Major to natural minor
        if new_key_type == 1:                    
            range_chain = itertools.chain(range(tonic_note - 8, 128, 12), range(tonic_note - 3, 128, 12), range(tonic_note - 1, 128, 12))
            for i in range_chain:
                if i >= 0:
                    module_tuples[i, 1] -= 1
        # Major to harmonic minor
        if new_key_type == 2:                    
            range_chain = itertools.chain(range(tonic_note - 8, 128, 12), range(tonic_note - 3, 128, 12))
            for i in range_chain:
                if i >= 0:
                    module_tuples[i, 1] -= 1
                    
    if prev_key_type == 1:
        # natural minor to Major
        if new_key_type == 0:
            range_chain = itertools.chain(range(tonic_note - 9, 128, 12), range(tonic_note - 4, 128, 12), range(tonic_note - 2, 128, 12))
            for i in range_chain:
                if i >= 0:
                    module_tuples[i, 1] += 1
        # natural minor to harmonic minor
        if new_key_type == 2:
            for i in range(tonic_note - 2, 128, 12):
                if i >= 0:
                    module_tuples[i, 1] += 1
        # harmonic minor to natural minor
        if new_key_type == 1:
            for i in range(tonic_note - 1, 128, 12):
                if i >= 0:
                    module_tuples[i, 1] -= 1
    
    # change tonic key
    module_tuples[:, 1] += ((new_key % 12 - prev_key % 12) + 6) % 12 - 6
                        
    return module_tuples[module_tuples[:, 1] != 0]
    
module_tuples = get_module_tuples(key_predict, 26)
print(module_tuples)

module_notes_dest_avail = np.ones(module_tuples.shape[0], dtype=np.bool)
module_notes_adapt = np.zeros(module_tuples.shape[0], dtype=np.bool)

# play audio
play_audio = True

midi_file_new = mido.MidiFile()
midi_tracks = midi_file.tracks

for track in midi_tracks:
    
    track_new = mido.MidiTrack()
    midi_file_new.tracks.append(track_new)
    
    for idx, msg in enumerate(track):
                
        if not msg.is_meta and is_note(msg) and not is_drum(msg):
            
            # update the availableness of current destinated notes
            if msg.note in (module_tuples[:, 0] + module_tuples[:, 1]):
                
                module_note_idx = np.nonzero(module_tuples[:, 0] + module_tuples[:, 1] == msg.note)[0][0]
                module_notes_dest_avail[module_note_idx] = not is_note_on(msg)
                
                
            # detected a note in the module tuples set
            if msg.note in module_tuples[:, 0]:
                module_note_idx = np.nonzero(module_tuples[:, 0] == msg.note)[0][0]
                module_note_dest = msg.note + module_tuples[module_note_idx, 1]
                
                module_append = False
                
                if is_note_on(msg):
                    
                    # decide module or not due to the current state of destinated note
                    if module_notes_dest_avail[module_note_idx]:

                        # destinated note is in 'note off' state
                        # decide module or not due to the existence of the destinated note before the 'note-off' of original note
                        idx_shift = idx
                        while True:

                            idx_shift += 1
                            IDX_SHIFT_MAX = 1000

                            if not track[idx_shift].is_meta and is_note(track[idx_shift]):

                                if idx_shift > len(track) or (idx_shift - idx) > IDX_SHIFT_MAX:
                                    print('WARNING: Auto while-loop break')
                                    break
                                
                                # reach the note-off message of original note, moduled note can be used
                                elif track[idx_shift].note == msg.note and not is_note_on(track[idx_shift]):
                                    # notify the note-off note to change
                                    module_notes_adapt[module_note_idx] = True
                                    module_append = True
                                    break
                                    
                                # reach the moduled note, moduled note cannot be used
                                elif track[idx_shift].note == module_note_dest:
                                    break
                
                else:
                    
                    if module_notes_adapt[module_note_idx]:
                        
                        module_notes_adapt[module_note_idx] = False
                        module_append = True
            
                if module_append:
                    track_new.append(msg.copy(note=module_note_dest))

                else:
                    track_new.append(msg)
                    
            else:

                track_new.append(msg)
                
        else:
            
            track_new.append(msg)
            
        
# use 'print(mido.get_output_names())' to get your port's name and type it in the 'mido.open_output' bracket
port = None
port = mido.open_output('Microsoft GS Wavetable Synth 0')

try:

    for msg in midi_file_new:
        
        # delay for playing
        if play_audio:
            time.sleep(msg.time)
            if not msg.is_meta:
                port.send(msg)

    if port is not None:
        port.close()
    print('End of midi')
    
# action after pressing stop
except KeyboardInterrupt:
    
    if port is not None:
        port.close()
    print('Forced stop')

[[  0   1]
 [ 12   1]
 [ 24   1]
 [ 36   1]
 [ 48   1]
 [ 60   1]
 [ 72   1]
 [ 84   1]
 [ 96   1]
 [108   1]
 [120   1]]
Forced stop


In [58]:
try:

    for msg in midi_file:
        
        # delay for playing
        if play_audio:
            time.sleep(msg.time)

        if msg.time != 0:

            total_duration += msg.time
            bar_duration += mido.second2tick(msg.time, ticks_per_beat, current_tempo)

            # indication of 'bar_duration' reaching the duration of a complete bar
            if bar_duration / ticks_per_beat >= time_signature_quarter:

                # break the continue-over-bars note-on notes
                notes_vec[np.where(notes_vec_on)] += total_duration
                
                # reset 'bar_duration'
                bar_duration_prev = bar_duration
                bar_duration = bar_duration - ticks_per_beat * time_signature_quarter * (bar_duration // (ticks_per_beat * time_signature_quarter))
                
                # find out the played notes of the complete bar
                played_notes_idx = np.where(notes_vec != 0)
                if played_notes_idx[0].size > 0:
                    played_notes = np.zeros(12, dtype=np.float16)
                    for played_note_idx in np.nditer(played_notes_idx):
                        played_notes[played_note_idx[1] % 12] += notes_vec[played_note_idx[0]][played_note_idx[1]]
                    played_notes = normalize(played_notes)
                                        
                    # give a better 'print' for played notes (all values are from 'played_notes')
                    played_notes_print = [np.where(played_notes != 0)[0], played_notes[played_notes != 0]]
                    played_notes_printf = ''
                    for played_note_print in np.nditer(played_notes_print):
                        played_notes_printf += '{:<2}'.format(note_int2char(played_note_print[0])) + ': ' + "{0:.2f}".format(played_note_print[1]) + '  '
                    
                    # calculate the chord similarity score
                    chords_score = np.dot(chords, played_notes)
                    chord_predict = np.argmax(chords_score)
                    chords_stat[chord_predict] += 1
                    
                    # print the chord analysis result
                    print('%s\t%s' % (name_chord(chord_predict), played_notes_printf))
                    
                    chords_all.append([total_duration, chord_predict])
                        
                # reset the notes_vec and notes_vec_on for next bar
                notes_vec = np.zeros([16, 128], dtype=np.float16)
                notes_vec_on = np.zeros([16, 128], dtype=np.bool)
                
                # continue the continue-over-bars-note-on notes
                notes_vec[np.where(notes_vec_on)] -= total_duration
                
                # debug message for bar detection
                # print('next bar:\t%.2f to %.2f' % (bar_duration_prev, bar_duration))

        if msg.is_meta:

            # change time signature should be the first beat
            if msg.type == 'time_signature':
                time_signature_denominator = msg.denominator
                time_signature_numerator = msg.numerator
                time_signature_quarter = time_signature_numerator / (time_signature_denominator / 4)
                bar_duration = 0
                print('time signature:\t%d/%d (quarters: %d)' % (time_signature_numerator, time_signature_denominator, time_signature_quarter))

            # update tempo for 'second2tick'
            if msg.type == 'set_tempo':
                current_tempo = msg.tempo
                print("set tempo:\t%d (bpm: %.2f)" % (msg.tempo, mido.tempo2bpm(msg.tempo)))

        if not msg.is_meta:

            # play the note
            if play_audio:
                port.send(msg)
            
            if msg.type == 'note_on':
                
                # ignore drum channel (midi channel = 10)
                if msg.channel != 9:
            
                    if msg.velocity != 0:
                        if not notes_vec_on[msg.channel][msg.note]:
                            notes_vec[msg.channel][msg.note] -= total_duration
                            notes_vec_on[msg.channel][msg.note] = True
                    else:
                        if notes_vec_on[msg.channel][msg.note]:
                            notes_vec[msg.channel][msg.note] += total_duration
                            notes_vec_on[msg.channel][msg.note] = False

            if msg.type == 'note_off':
                
                # ignore drum channel (midi channel = 10)
                if msg.channel != 9:
                    
                    if notes_vec_on[msg.channel][msg.note]:
                        notes_vec[msg.channel][msg.note] += total_duration
                        notes_vec_on[msg.channel][msg.note] = False

        # you can set auto stop in here (seconds as unit)
        if total_duration == -1:

            if port is not None:
                port.close()
            break

    if port is not None:
        port.close()
    print('End of midi')
    key_stat = np.dot(chords_stat[24:], chord_to_key)
    key_predict = np.argmax(key_stat)
    key_acc = np.sum(chords_stat[np.add(key_to_chords(key_stat), 24)]) / np.sum(chords_stat[24:])
    print('\nKey: %s (accuracy: %f)' % (name_key(key_predict), key_acc))
    print('\nChord statistics')
    for chord_stat_idx, chord_stat in enumerate(chords_stat):
        if chord_stat != 0:
            print('%s\t: %d' % (name_chord(chord_stat_idx), chord_stat))
        
# action after pressing stop
except KeyboardInterrupt:
    
    if port is not None:
        port.close()
    print('Forced stop')

'D minor'

In [29]:
# tracks_rhythm_pattern = np.zeros([len(midi_tracks), ])



ticks_per_beat = midi_file.ticks_per_beat

cumulative_tick = 0
total_time = 0
shift_tick = 0
bar_time = []

for msg in midi_file:
    
    if msg.time != 0:
        cumulative_tick += mido.second2tick(msg.time, ticks_per_beat, current_tempo)
        total_time += msg.time
        
    
    if msg.is_meta:

        # change time signature should be the first beat
        if msg.type == 'time_signature':
            time_signature_denominator = msg.denominator
            time_signature_numerator = msg.numerator
            time_signature_quarter = time_signature_numerator / (time_signature_denominator / 4)
            bar_duration = 0
            print('time signature:\t%d/%d (quarters: %d)' % (time_signature_numerator, time_signature_denominator, time_signature_quarter))
            print(cumulative_tick / ticks_per_beat)

        # update tempo for 'second2tick'
        if msg.type == 'set_tempo':
            current_tempo = msg.tempo
            print("set tempo:\t%d (bpm: %.2f)" % (msg.tempo, mido.tempo2bpm(msg.tempo)))
            print(cumulative_tick / ticks_per_beat)
            
            cumulative_bar = cumulative_tick / ticks_per_beat / time_signature_quarter
            print(cumulative_bar)
            
            while True:
                shift_tick += 1
                if cumulative_bar + 1 > bar_count:
                    bar_time.append(bar_count)
                else:
                    break
            
            cumulative_tick = 0

cumulative_bar = cumulative_tick / ticks_per_beat / time_signature_quarter
print(cumulative_bar)

bar_count = shift_tick
while True:
    if cumulative_bar + 1 > bar_count:
        bar_time.append(bar_count)
    else:
        break
    bar_count += 1

cumulative_tick = 0
shift_tick = total_time
                
print(bar_time)


set tempo:	472440 (bpm: 127.00)
0.0
0.0


NameError: name 'bar_count' is not defined

In [94]:
# get the time and tick indicating the bars
def get_bars_info(midi_file):
    ticks_per_beat = midi_file.ticks_per_beat

    total_tick = 0
    total_time = 0
    rhythm_events = []

    # store the time signature and tempo change events as rhythm_events
    for msg in midi_file:

        # sum up the tick/time
        if msg.time != 0:
            total_tick += mido.second2tick(msg.time, ticks_per_beat, current_tempo)
            total_time += msg.time

        # store meta message of time signature and tempo with current tick
        if msg.is_meta:

            if msg.type == 'time_signature':
                time_signature_quarter = msg.numerator / (msg.denominator / 4)
                rhythm_events.append([total_tick, -1, time_signature_quarter])

            if msg.type == 'set_tempo':
                current_tempo = msg.tempo
                rhythm_events.append([total_tick, current_tempo, -1])

    rhythm_events.append([total_tick, -1, -1])

    cumulative_time = 0
    cumulative_tick = 0
    prev_tick = 0
    current_tempo = -1
    current_ts_quarter = -1
    bars_time = []
    bars_tick = []

    # append the time/tick at each bar according to 'rhythm_events'
    for rhythm_event in rhythm_events:

        # detect rhythm change, then conclude the bars before the change
        if rhythm_event[0] != prev_tick:

            tick_diff = rhythm_event[0] - prev_tick
            prev_tick = rhythm_event[0]

            # conclude the bar time/tick by dividing the bar length
            num_of_bar = tick_diff / ticks_per_beat
            num_of_bar_2 = num_of_bar
            while num_of_bar > 0:
                bar_tick = (num_of_bar_2 - num_of_bar) * ticks_per_beat
                bars_time.append(mido.tick2second(bar_tick, ticks_per_beat, current_tempo) + cumulative_time)
                bars_tick.append(bar_tick + cumulative_tick)
                num_of_bar -= current_ts_quarter            

            cumulative_time += mido.tick2second(tick_diff, ticks_per_beat, current_tempo)
            cumulative_tick += tick_diff

        # update the time signature and time tempo events
        if rhythm_event[1] != -1:
            current_tempo = rhythm_event[1]
        if rhythm_event[2] != -1:
            current_ts_quarter = rhythm_event[2]

    return bars_time, bars_tick

def get_channel_info(midi_file):
    channel_count = np.zeros(16)
    for msg in midi_file:
        if not msg.is_meta and is_note(msg):
            channel_count[msg.channel] += 1
    return channel_count

In [73]:
def get_bars_note_info(midi_file):

    _, bars_tick = get_bars_info(midi_file)

    ticks_per_beat = midi_file.ticks_per_beat
    current_tempo = -1
    current_ts_quarter = -1

    total_tick = 0
    bar_start_tick = bars_tick.pop(0)
    bar_note_info = []
    bars_note_info = []

    # store the time signature and tempo change events as rhythm_events
    for msg in midi_file:

        # sum up the tick/time
        if msg.time != 0:        
            total_tick += mido.second2tick(msg.time, ticks_per_beat, current_tempo)
            if len(bars_tick) != 0 and total_tick >= bars_tick[0]:

                # find the suitable bar for placing current note
                bars_shift_idx = 0
                while len(bars_tick) > (bars_shift_idx + 1) and total_tick >= bars_tick[bars_shift_idx + 1]:
                    bars_shift_idx += 1

                bars_note_info.append([bar_start_tick, bars_tick[0] - bar_start_tick, current_ts_quarter, current_tempo, bar_note_info])
                bar_note_info = []
                bar_start_tick = bars_tick.pop(0)

                # put back the shifted (empty) bars
                for pad_bar_idx in range(bars_shift_idx):
                    bars_note_info.append([bar_start_tick, bars_tick[0] - bar_start_tick, current_ts_quarter, current_tempo, bar_note_info])
                    bar_note_info = []
                    bar_start_tick = bars_tick.pop(0)


        # store meta message of time signature and tempo with current tick
        if msg.is_meta:

            if msg.type == 'time_signature':
                current_ts_quarter = msg.numerator / (msg.denominator / 4)

            if msg.type == 'set_tempo':
                current_tempo = msg.tempo

        if not msg.is_meta:

            if is_note(msg):

                if is_note_on(msg):
                    bar_note_info.append([msg.channel, msg.note, total_tick, 1])

                else:
                    bar_note_info.append([msg.channel, msg.note, total_tick, 0])
    return bars_note_info

In [142]:
# input the file name
midi_string = 'sample4'
midi_file = mido.MidiFile('%s.mid' % midi_string)

bars_note_info = get_bars_note_info(midi_file)

MAX_NUM_OF_CHANNELS = 16
MAX_NUM_OF_NOTE_IN_BAR = 64
tracks_rhythm_pattern = np.zeros([MAX_NUM_OF_CHANNELS, len(bars_note_info), MAX_NUM_OF_NOTE_IN_BAR, 2]) - 1
# print(tracks_rhythm_pattern.shape)


prev_bar_notes = []

for idx, bar_note_info in enumerate(bars_note_info):
    
    if len(prev_bar_notes) != 0:
        for prev_bar_note in prev_bar_notes:
            append_idx = 0
            while append_idx < 64:
                if tracks_rhythm_pattern[prev_bar_note[0], idx, append_idx, 0] == -1:
                    tracks_rhythm_pattern[prev_bar_note[0], idx, append_idx] = [0, prev_bar_note[1]]
                    break
                append_idx += 1
        prev_bar_notes = []
    
    # print(idx, bar_note_info[4])
    for note in bar_note_info[4]:
        if note[3]:
            append_idx = 0
            while append_idx < 64:
                if tracks_rhythm_pattern[note[0], idx, append_idx, 0] == -1:
                    tracks_rhythm_pattern[note[0], idx, append_idx, 0] = note[2] - bar_note_info[0]
                    tracks_rhythm_pattern[note[0], idx, append_idx, 1] = -2 - note[1]
                    break
                append_idx += 1
        else:
            # print(tracks_rhythm_pattern[note[0], idx, :, 1], -2 - note[1])
            relative_idx = np.nonzero(tracks_rhythm_pattern[note[0], idx, :, 1] == -2 - note[1])[0][0]
            tracks_rhythm_pattern[note[0], idx, relative_idx, 1] = note[2] - bar_note_info[0] - tracks_rhythm_pattern[note[0], idx, relative_idx, 0]
            # print(tracks_rhythm_pattern[note[0], idx, relative_idx])
    
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        
        temp_idx = 0

        while temp_idx < MAX_NUM_OF_NOTE_IN_BAR and tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1] != -1:
            if tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1] < -1:
                prev_bar_notes.append([channel_idx, tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1]])
                tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1] = bar_note_info[1] - tracks_rhythm_pattern[note[0], idx, relative_idx, 0]
                # print(tracks_rhythm_pattern[channel_idx, idx, temp_idx])
            tracks_rhythm_pattern[channel_idx, idx, temp_idx] /= bar_note_info[1]
            temp_idx += 1
    # print(prev_bar_notes)
        
tracks_rhythm_pattern[0]

array([[[-1.        , -1.        ],
        [-1.        , -1.        ],
        [-1.        , -1.        ],
        ...,
        [-1.        , -1.        ],
        [-1.        , -1.        ],
        [-1.        , -1.        ]],

       [[-1.        , -1.        ],
        [-1.        , -1.        ],
        [-1.        , -1.        ],
        ...,
        [-1.        , -1.        ],
        [-1.        , -1.        ],
        [-1.        , -1.        ]],

       [[-1.        , -1.        ],
        [-1.        , -1.        ],
        [-1.        , -1.        ],
        ...,
        [-1.        , -1.        ],
        [-1.        , -1.        ],
        [-1.        , -1.        ]],

       ...,

       [[ 0.        ,  0.16666667],
        [ 0.        ,  0.22916667],
        [ 0.16666667,  0.08333333],
        ...,
        [-1.        , -1.        ],
        [-1.        , -1.        ],
        [-1.        , -1.        ]],

       [[ 0.        ,  0.25      ],
        [ 0.        ,  0.22

In [124]:
# input the file name
midi_string = 'sample1'
midi_file = mido.MidiFile('%s.mid' % midi_string)
midi_tracks = midi_file.tracks

channel_count = np.zeros(16)


for msg in midi_file:
    if not msg.is_meta and is_note(msg):
        if msg.note == 38 and msg.channel == 1:
            print(msg)
        

note_on channel=1 note=38 velocity=88 time=0
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.0029527499999999996
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on c

note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 velocity=88 time=0
note_off channel=1 note=38 velocity=80 time=0.033464499999999994
note_on channel=1 note=38 vel